In [40]:
import pandas as pd
import numpy as np
import os


# Load data
current_folder = globals()['_dh'][0]
minute_prices = pd.read_csv(os.path.join(current_folder, 'minute_prices.csv'))
imbalance_prices = pd.read_csv(os.path.join(current_folder, 'imbalance_prices.csv'))

minute_prices = minute_prices[['datum', 'tijd', 'Hoogste_prijs_opregelen', 'Laagste_prijs_afregelen']]
minute_prices = minute_prices.rename(columns={'datum': 'date', 'tijd': 'time', 'Hoogste_prijs_opregelen': 'high_take_price', 'Laagste_prijs_afregelen': 'low_feed_price'})

print(minute_prices.shape)
print(imbalance_prices.shape)
print(minute_prices['date'][0])

minute_prices.insert(4, "medium_price", np.zeros(minute_prices.shape[0]), True)
minute_prices.insert(5, "imbalance_take_price", np.zeros(minute_prices.shape[0]), True)
minute_prices.insert(6, "imbalance_feed_price", np.zeros(minute_prices.shape[0]), True)
minute_prices.insert(7, "imbalance_regulation_state", np.zeros(minute_prices.shape[0]), True)


j = -1
for i in range (minute_prices.shape[0]):
  if i % 15 == 0:
    j+=1
  afnemen = round(np.float32(imbalance_prices['Afnemen'][j].replace(',','.')), 2)
  invoeden = round(np.float32(imbalance_prices['invoeden'][j].replace(',','.')), 2)
  regeltoestand = round(np.int64(imbalance_prices['regeltoestand'][j]), 2)
  if regeltoestand != 2:
    half_price = (afnemen / 2.0)
  else:
    half_price = (afnemen + invoeden) / 2.0

  high_take_price = minute_prices['high_take_price'][i]
  low_feed_price = minute_prices['low_feed_price'][i]

  if not pd.isnull(high_take_price):
    minute_prices.loc[i, 'high_take_price'] = round(np.float32(minute_prices['high_take_price'][i].replace(',', ".")),2)

  if not pd.isnull(low_feed_price):
    minute_prices.loc[i, 'low_feed_price'] = round(np.float32(minute_prices['low_feed_price'][i].replace(',', ".")),2)

  minute_prices.loc[i, 'imbalance_take_price'] = afnemen
  minute_prices.loc[i, 'imbalance_feed_price'] = invoeden
  minute_prices.loc[i, 'medium_price'] = half_price
  minute_prices.loc[i, 'imbalance_regulation_state'] = regeltoestand

minute_prices['imbalance_take_price'] = minute_prices['imbalance_take_price'].round(2)
minute_prices['imbalance_feed_price'] = minute_prices['imbalance_feed_price'].round(2)
minute_prices['medium_price'] = minute_prices['medium_price'].round(2)

minute_prices.to_csv(os.path.join(current_folder, 'new_imb.csv'))


(525600, 4)
(35040, 12)
01-01-2023


In [10]:
import pandas as pd
import numpy as np
import os

# Load data
current_folder = globals()['_dh'][0]
clean_imb = pd.read_csv(os.path.join(current_folder, 'imb_clean.csv'))

clean_imb.insert(2, "day_of_year", np.zeros(clean_imb.shape[0]), True)
clean_imb.insert(3, "hour_of_day", np.zeros(clean_imb.shape[0]), True)

clean_imb['date'] = pd.to_datetime(clean_imb['date'], format='%d-%m-%Y')
clean_imb['day_of_year'] = clean_imb['date'].dt.dayofyear
clean_imb['time'] = pd.to_datetime(clean_imb['time'], format='%H:%M')
clean_imb['hour_of_day'] = clean_imb['time'].dt.hour

clean_imb.to_csv(os.path.join(current_folder, 'imb_clean.csv'))


In [11]:
import pandas as pd
import numpy as np
import os

# Load data
current_folder = globals()['_dh'][0]
clean_imb = pd.read_csv(os.path.join(current_folder, 'imb_clean.csv'))
minute_prices = pd.read_csv(os.path.join(current_folder, 'minute_prices.csv'))

clean_imb['time'] = minute_prices['tijd']

clean_imb.to_csv(os.path.join(current_folder, 'imb_clean.csv'))

In [15]:
import pandas as pd
import numpy as np
import os

# Load data
current_folder = globals()['_dh'][0]
clean_imb = pd.read_csv(os.path.join(current_folder, 'imb_clean.csv'))

df = clean_imb.loc[:, ~clean_imb.columns.str.contains('^Unnamed')]

df.head()
df.to_csv(os.path.join(current_folder, 'imb_clean.csv'))

In [2]:
import pandas as pd
import numpy as np
import os

# Load data
current_folder = globals()['_dh'][0]
clean_imb = pd.read_csv(os.path.join(current_folder, 'imb_clean.csv'))
clean_imb = clean_imb.rename(columns={'imbalance_take_price': 'imbalance_feed_price', 'imbalance_feed_price': 'imbalance_take_price'})
clean_imb.to_csv(os.path.join(current_folder, 'imb_clean.csv'))